# Return Attribution Analysis

This notebook demonstrates return attribution by asset type and region using Polars.

In [ ]:
import os
import polars as pl
import matplotlib.pyplot as plt


## 1. Load Data


In [ ]:
datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
tx = pl.read_csv(f'{datasets_dir}/cleaned_transactions.csv')
assets = pl.read_parquet(f'{datasets_dir}/assets.parquet')
benchmarks = pl.read_csv(f'{datasets_dir}/benchmarks.csv')


## 2. Join and Trade Value Calculation


In [ ]:
df = tx.join(assets, on='asset_id', how='inner')
df = df.with_columns([(pl.col('amount') * pl.col('price')).alias('trade_value')])


## 3. Attribution by Asset Type


In [ ]:
asset_type_attr = (
    df.group_by('asset_type')
      .agg([pl.sum('trade_value').alias('total_value'), pl.count().alias('num_trades')])
)
total = asset_type_attr['total_value'].sum()
asset_type_attr = asset_type_attr.with_columns([(pl.col('total_value') / total * 100).alias('pct_attribution')])
print('Return attribution by asset type:')
print(asset_type_attr)


## 4. Attribution by Region


In [ ]:
region_attr = (
    df.group_by('region')
      .agg([pl.sum('trade_value').alias('total_value'), pl.count().alias('num_trades')])
)
region_attr = region_attr.with_columns([(pl.col('total_value') / total * 100).alias('pct_attribution')])
print('Return attribution by region:')
print(region_attr)


## 5. Save Results


In [ ]:
asset_type_attr.write_csv(f'{datasets_dir}/asset_type_attribution.csv')
region_attr.write_csv(f'{datasets_dir}/region_attribution.csv')
print('Saved attribution results to CSV.')
